In [ ]:
# Install necessary libraries for language model fine-tuning and inference
!pip install unsloth  # Core library for model management
!pip install torch  # PyTorch library for GPU-accelerated training and inference
!pip install transformers  # Hugging Face library for NLP model handling
!pip install datasets  # Library for loading and managing datasets
!pip install trl  # Library for training reinforcement learning-based NLP models


In [ ]:
# Import required modules for model management, dataset handling, and fine-tuning
import torch
from unsloth import FastLanguageModel  # High-performance language model utilities
from unsloth.chat_templates import get_chat_templatefrom datasets import load_dataset  # For loading datasets
from trl import SFTTrainer  # Supervised fine-tuning trainer
from transformers import TrainingArguments  # Configuration for training process
from unsloth.chat_templates import get_chat_template, standardize_sharegpt  # Chat template utilities


In [ ]:
# Load a pre-trained Llama-3.2 model with 3 billion parameters, optimized for instruction following
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-3B-Instruct",
    max_seq_length=2048,
    load_in_4bit=True,  # Use 4-bit precision for efficient memory usage
)


In [ ]:
# Apply Parameter-Efficient Fine-Tuning (PEFT) to reduce training resource requirements
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Bottleneck dimension for fine-tuning
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",  # Attention projection layers
        "gate_proj", "up_proj", "down_proj",  # Feedforward layers
    ],
)


In [ ]:
# Configure tokenizer with chat-style templates for input-output formatting
tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")

# Load the dataset and select the training split
dataset = load_dataset("mlabonne/FineTome-100k", split="train")

# Standardize the dataset using ShareGPT format and prepare input text using templates
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(
    lambda examples: {
        "text": [
            tokenizer.apply_chat_template(convo, tokenize=False)
            for convo in examples["conversations"]
        ]
    },
    batched=True,  # Process in batches for efficiency
)


In [ ]:
# Configure the fine-tuning trainer with training arguments
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    args=TrainingArguments(
        per_device_train_batch_size=2,  # Batch size per device
        gradient_accumulation_steps=4,  # Accumulate gradients over 4 steps
        warmup_steps=5,  # Warm-up steps for learning rate
        max_steps=60,  # Total training steps
        learning_rate=2e-4,  # Learning rate
        fp16=not torch.cuda.is_bf16_supported(),  # Use FP16 if BF16 not supported
        bf16=torch.cuda.is_bf16_supported(),  # Use BF16 if supported
        logging_steps=1,  # Log every step
        output_dir="outputs",  # Directory for saving outputs
    ),
)


In [ ]:
# Perform fine-tuning on the model
trainer.train()

# Save the fine-tuned model to the specified directory
model.save_pretrained("finetuned_model")


In [ ]:
# Load the fine-tuned model for inference
model_path = "finetuned_model"  # Directory of the fine-tuned model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_path,
    max_seq_length=2048,
    load_in_4bit=True,
)


In [ ]:
# Enable faster inference using optimized settings
FastLanguageModel.for_inference(model)


In [ ]:
# Import required modules for inference
import torch
from unsloth import FastLanguageModel

# Load the fine-tuned model and tokenizer
model_path = "finetuned_model"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_path,
    max_seq_length=2048,
    load_in_4bit=True,  # Use 4-bit precision
)

# Enable optimized inference
FastLanguageModel.for_inference(model)

# Set the model to evaluation mode
model.eval()

# Determine the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move the model to the appropriate device

# Define an input prompt for text generation
input_prompt = """I am 25 years old, earning $50,000 per year. I have $5,000 in credit card 
    debt at 18% interest, $10,000 in student loans at 5% interest, and $2,000 in savings. 
    I want to buy a house within the next 5 years, but I also need to save for retirement. 
    How should I prioritize paying off my debt, saving for a down payment, and investing 
    for retirement?"""

# Tokenize the input prompt with padding and truncation
inputs = tokenizer(
    input_prompt,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=2048
).to(device)

# Ensure model and inputs are on the same device
assert model.device == inputs["input_ids"].device, "Model and inputs are on different devices."

# Generate text without gradient computation
with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=200,  # Maximum length of generated text
        num_return_sequences=1,  # Generate a single output sequence
        temperature=0.7,  # Sampling temperature
        top_p=0.9,  # Top-p nucleus sampling
    )

# Decode and print the generated text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)
